In [ ]:
from competition_utils import *

# 제출 함수를 리스트로 정의 (submission_N_학번)
# 깃허브에는 최대 3개 업로드 데모시 하이퍼파라미터 튜닝 후 1개만 제출
submission_functions = ['submission_1_20223507', 'submission_2_20223507', 'submission_3_20223507']  
for submission_function in submission_functions:
    exec(f"from {submission_function} import {submission_function}")

# 분석 방향에 따라 iteration 수 수정
# 평가시에는 데모와 다른 스플릿 시드를 만들어 [1, 10]로 진행 예정
iterations = [1, 10] 

# 'Crown Detection'는 예제 데이터셋, 데모 및 평가시에는 'CV_Competition'
Dataset_Name = 'CV_Competition' 

# 결과를 모아 하나의 csv 파일에 저장
results_df = pd.DataFrame(columns=[
    'Experiment Time', 'Iteration', 'Submission Function', 
    'IoU', 'Dice', 'Precision', 'Recall', 'Output Json Path',
])
csv_filename = f"Evaluation_Results_{datetime.now().strftime('%y%m%d_%H%M%S')}.csv"

# 서로 다른 iteration, submission function으로 실험 진행
for iteration in range(iterations[0], iterations[1]+1):
    yaml_path = f'Datasets/{Dataset_Name}/data_iter_{iteration:02d}.yaml'
    for submission_function in submission_functions:
        ex_time = datetime.now().strftime('%y%m%d_%H%M%S')
        output_json_path = f"{ex_time}_{submission_function}_Iter_{iteration}_detection_results.json"
        globals()[submission_function](yaml_path, output_json_path)
        labels_dir = f"Datasets/{Dataset_Name}/labels"  
        vis_output_dir = f"{ex_time}_visualization_results"  
        image_level_result_path =f"{ex_time}_{submission_function}_Iter_{iteration}_image_level_results.json"
        stats = eval_and_vis(yaml_path, output_json_path, labels_dir, image_level_result_path, vis_output_dir, vis=False) # 분석 방향에 따라 vis=True 설정
        new_row = {
            'Experiment Time': ex_time,
            'Iteration': iteration,
            'Submission Function': submission_function,
            'IoU': stats['IoU']['avg'],
            'Dice': stats['Dice']['avg'],
            'Precision': stats['Precision']['avg'],
            'Recall': stats['Recall']['avg'],
            'Output Json Path': output_json_path,
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        results_df.to_csv(csv_filename, index=False)

[Offline 증강] 데이터가 이미 증강되었습니다.


KeyboardInterrupt: 

In [ ]:
# 분석 방향에 따라 전체 결과를 모아 평균-표준편차-통계테스트 결과 저장 

keep_columns = ['Iteration','Submission Function']
keep_measures = ['IoU','Dice','Precision','Recall']  

reduction = 'Iteration'
row = 'Measure Type'
column = 'Submission Function'
reference_column = 'submission_1_20223507'

custom_fmt_template = '{mean_fmt} ± {std_fmt} {significance}'
significance_levels = [0.1, 0.05, 0.01]
decimal_places = 3
transpose = True
make_tables(csv_filename, keep_columns, keep_measures, reduction, row, column,  
            reference_column, custom_fmt_template, significance_levels, decimal_places, transpose)